In [7]:
##install modules and import definitions

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install matplotlib
!pip install pandas
!pip install openpyxl
# !pip install pyperclip
# !pip install pywinauto
!pip install xlsxwriter
import tqdm
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import time
# import pyperclip
import os
from datetime import datetime
import xlsxwriter
# import pywinauto

def login(Id, password,driver):
    #login at mobile emulater'https://www.instagram.com/'
    #input : Id and password of user, and driver which is webdriver object
    #output : None
    driver.implicitly_wait(4)
    driver.find_element(By.XPATH, '//*[@id="id"]').send_keys(Id)
    driver.find_element(By.XPATH, '//*[@id="pw"]').send_keys(password) 
                                                    
    click_to_login = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="col_pw"]/div')))
    click_to_login.click()
    
    # 중앙대 요금 미납 관련
    try :
        driver.find_element(By.XPATH, '//*[@id="column_5"]/div/div[10]').click()
    except :
        pass
    
def get_back(driver):
    driver.find_element(By.XPATH, '//*[@id="top_btns"]/p[1]/a/img').click()

def set_period(daystart, dayend, driver, change_month=None): ##Added on 2.24
#    need to implement before getting table both in get_by_
#    input :
#        no change if value is -1 both for daystart and dayend
#        daystart(num) : start day to search, 2 if you want to get date 2 on this month
#        dayend(num) : end day to search
#        driver : webdriver

    def click_date(day_to_click,driver) :
        #  input : num (day)
        #  output : None, select day
        #  driver : webdriver
        time.sleep(1)
        startcalendar = driver.find_elements_by_class_name('ui-state-default')
        time.sleep(3)
        for day in startcalendar :
          if int(day.text) == day_to_click :
              day.click()
              break

    if daystart != 0 :
        if change_month is not None :
          previous_arrow = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ui-datepicker-div"]/div/a[1]')))
          previous_arrow.click()
          time.sleep(1)
        datepicker_start = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="datepicker2"]')))
        datepicker_start.click()
        click_date(daystart, driver)
    if dayend != 0 :
        if change_month is not None :
          previous_arrow = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="ui-datepicker-div"]/div/a[1]')))
          previous_arrow.click()
          time.sleep(1)
        datepicker_end = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="datepicker3"]')))
        datepicker_end.click()
        driver.implicitly_wait(6)
        click_date(dayend, driver)
    submit = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="sub1_btns"]/p[1]/a/img')))
    submit.click()
    driver.implicitly_wait(4)
    
def click_get_by_goods(driver): ##Added on 2.24
    
    click_to_ = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="tb_mainicon"]/tbody/tr[2]/td[3]/a/img')))
    click_to_.click()
    
def get_by_goods(driver): ##Updated on 22.2.24
    driver.implicitly_wait(4)
    table = driver.find_element_by_class_name('sub_tb2_2')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_tag_name("tr")

    row_index=[0]*len(rows)
    col_index = ['분류명', '상품명','가격','건수','합계']
    
    table_array = np.array([[None]*len(col_index) for _ in range(len(row_index))])
    
    for index, value in enumerate(rows):
        body=value.find_elements_by_tag_name("td")
        table_array[index,:] = [element.text for element in body]
    
    return table_array

def click_get_by_time(driver): ##Added on 22.2.24
    
    click_to_ = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="tb_mainicon"]/tbody/tr[1]/td[3]/a/img')))
    click_to_.click()
    
    
def get_by_time(driver): ##Updated on 22.2.24
    
    driver.implicitly_wait(4)
    
    table = driver.find_element_by_class_name('sub_tb2_2')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_tag_name("tr")

    row_index=[0]*len(rows)
    col_index=['판매시간', '매출합계', '건수', '객수']
    
    table_array = np.array([[None]*len(col_index) for _ in range(len(row_index))])
   
    for index, value in enumerate(rows):
        body=value.find_elements_by_tag_name("td")
        table_array[index,:] = [element.text for element in body]
    return table_array

def fine_data(info_by_goods,name):
    
    # input : np.array of df by goods where one shop
    # output : preprocess of data which is useless( option, etc)
    #         and add rows of same index, and drop all duplicated rows by index
    #         and output fined, sorted dataframe
    #updated(22.02.22) - daily sell divided into delievery or not
    list_to_del=[]
    delievery_company_list = ['쿠팡이츠 매출등록_','쿠팡이츠 매출등록','요기요 매출등록_','요기요 매출등록' ,'배달의민족 매출등록_',
                              '배달의민족 매출등록','헝그리판다 매출등록_','헝그리판다 매출등록']

    for index in range(len(info_by_goods)) :
        if info_by_goods[index][-1]== '0':
            list_to_del.append(index)
        elif info_by_goods[index][0] == '' :
            list_to_del.append(index)
        elif info_by_goods[index][0] == '소계' :
            list_to_del.append(index)

    fined = np.delete(info_by_goods, list_to_del, axis=0)

    temp=pd.DataFrame(fined[:-1,2:],index=fined[:-1,1],columns = ['가격','수량','소계'])

    for col_idx in range(len(temp.columns)):
        temp[temp.columns[col_idx]] = temp[temp.columns[col_idx]].str.replace(',', '').astype('int64')

    for index in temp.index :
        if temp.index.value_counts().loc[index]>1:
            temp.loc['{}_'.format(index)] = temp.loc[index].sum(axis=0)

    fined_df = temp.loc[~temp.index.duplicated(keep=False)]
    delievery_sum = sum([fined_df.loc[i, '소계'] for i in delievery_company_list if i in fined_df.index])
    daily_sum = fined_df['소계'].sum()
    fined_df.loc['배달/현장/합계'] = [delievery_sum,daily_sum-delievery_sum,daily_sum]
    sorted_df = fined_df.sort_values(by='수량', ascending=False)
    sorted_df.index.name = name
    sorted_df.reset_index(level=0,inplace=True)
    
    return sorted_df

def make_filename (daystart, dayend) : #Added on 22.02.24
  ##input : (num, num)
  ##output : filename, if input is 0, it automatcally gives value as today's date
  ##error : if daystart is bigger than dayend, it triggers error
  today = datetime.today().strftime("%Y%m%d%H%M%S")[:8]
  nowdate = today[-2:]
  daystart = daystart if daystart else nowdate
  dayend = dayend if dayend else nowdate
  if int(daystart) <10 :
    daystart = '0' +str(daystart)
  if int(dayend) <10 :
    dayend = '0' + str(dayend)
  if int(daystart) > int(dayend) :
    raise Exception("daystart can never bigger than dayend")
  if int(daystart) > int(nowdate) or int(dayend) > int(nowdate) :
    raise Exception("Range can't bigger than today's date which is day {}".format(nowdate))
  filename = today[:-2] + str(daystart) + '-' + str(dayend)

  return filename

def logout():
    logout = driver.find_element_by_xpath('//*[@id="top_btns"]/p[3]/a')
    logout.click()
    time.sleep(0.5)
    
    for _ in range(2):
        driver.switch_to.alert.accept()
def chromeSetting():
    ## setting chrome options for colab env
    ## return : Setted driver for colab
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    return driver


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [2]:
########main code###########
############################

#######variables to input#
#(daystart, dayend) = (num, num), if 0 : default value which is today
# onlyschool = (bool) if True : only search near school shop, else search all shops
daylist = range(1,23)
onlyschool = True

for day in daylist :
  daystart, dayend = day, day ##Added on 2.24###
  Id = []
  pwd= []
  name =[]
  url = []

  info_by_time = [None]*len(Id)
  info_by_goods= [None]*len(Id)
  df_by_goods = [None]*len(Id)

  today = datetime.today().strftime("%Y%m%d%H%M%S")[:8]
  base_dir = os.getcwd()
  # os.chmod('/Users/gwihwango/Desktop/python project/chromedriver', 0o755) ##Added for Mac
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  driver.get(url)
  # search_range = 9 if onlyschool else len(Id)
  search_range = 1
  for i in range(search_range):
      
      login(Id[i],pwd[i],driver)

      #get info 시간대별
      click_get_by_time(driver)
      set_period(daystart, dayend, driver)
      info_by_time[i] = get_by_time(driver)
      time.sleep(2)
      for j in range(2) :##Added on 2.24
          get_back(driver)
          driver.implicitly_wait(1)

      #get info 상품별  
      click_get_by_goods(driver)
      set_period(daystart, dayend, driver)##Added on 2.24
      driver.implicitly_wait(3)
      info_by_goods[i] = get_by_goods(driver)
      df_by_goods[i] = fine_data(info_by_goods[i],name[i])
      info_by_time[i][0][0]=name[i]
      for j in range(2) :##Added on 2.24
          get_back(driver)
          driver.implicitly_wait(1)
      print('The {} store is completed\n'.format(name[i]))

      logout()
    
  #Concatenate two DataFrames into one
      
  df_by_time = pd.DataFrame(pd.concat([pd.DataFrame(df) for df in info_by_time], axis=1))
  df_by_goods = pd.concat(df_by_goods, axis=1)

  ##Make Excel File from DataFrame

  file_name = 'infoBB_{}.xlsx'.format(make_filename (daystart,dayend))
  xlxs_dir =os.path.join(base_dir,file_name)
  writer = pd.ExcelWriter(xlxs_dir, engine='xlsxwriter')

  df_by_time.to_excel(writer, sheet_name='Sheet1')
  df_by_goods.to_excel(writer, sheet_name='Sheet2')

  writer.save()
  print(f"file saved as {xlxs_dir}")
  ##send_email - needed fixing
  # send_email(file_name, xlxs_dir,'wuzhensj@naver.com')

print('######################################')
print(f"ALL_COMPLETED")
print('######################################')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:122: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: DeprecationWarning: find_elements_by_tag_name is depreca

The ewha store is completed



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:196: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


file saved as /content/infoBB_20220501-01.xlsx


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:122: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: DeprecationWarning: find_elements_by_tag_name is depreca

...

The ewha store is completed



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:196: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


file saved as /content/infoBB_20220518-18.xlsx


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:122: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: DeprecationWarning: find_elements_by_tag_name is depreca

KeyboardInterrupt: ignored

In [4]:
file_name = 'infoBB_{}.xlsx'.format('all')
xlxs_dir =os.path.join(base_dir,file_name)
writer = pd.ExcelWriter(xlxs_dir, engine='xlsxwriter')

df_by_time.to_excel(writer, sheet_name='Sheet1')
df_by_goods.to_excel(writer, sheet_name='Sheet2')

writer.save()
print(f"file saved as {xlxs_dir}")

file saved as /content/infoBB_all.xlsx
